In [58]:
!pip install Korpora
!pip install seqeval
!pip install transformers

In [4]:
from Korpora import Korpora

Korpora.fetch("naver_changwon_ner")
corpus = Korpora.load("naver_changwon_ner")

[naver_changwon_ner] download train_data: 16.9MB [00:00, 98.8MB/s]                            



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : 네이버 + 창원대
    Repository : https://github.com/naver/nlp-challenge/tree/master/missions/ner
    References : http://air.changwon.ac.kr/?page_id=10

    개체명(Named Entity)은 인명, 기관명, 지명 등과 같이 문장 또는 문서에서 특정한 의미를 가지고 있는 단어 또는 어구를 말합니다.
    이 때문에 개체명은 정보 검색 및 언어 이해를 위한 분석에서 주요한 대상으로 다루어지고 있습니다.
    Data.ly에서는 개체명 코퍼스를 제공하여 연구에 도움을 드리고자 하며, 공개적인 리더보드를 통해 많은 분들의 연구 동향을 논의/공유하고자 합니다.
    제공되는 코퍼스는 Data.ly에서 제작한 것으로, 연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

    # License
    연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

[Korpora] Corpus `naver_changwon_ner` is already installed at /root/Korpora/naver_changwon_ner/train_data


In [44]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from tensorflow.keras.optimizers import SGD
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split
from transformers import shape_list, BertTokenizer, TFBertModel, AdamW
import tensorflow as tf

from seqeval.metrics import f1_score, classification_report

In [6]:
df = pd.DataFrame()
df['text'] = corpus.train.texts
df['words'] = corpus.train.words
df['tags'] = corpus.train.tags

In [7]:
all_tag = []
for tags in tqdm(corpus.train.tags):
  for tag in tags:
    if tag not in all_tag:
      all_tag.append(tag)


100%|██████████| 90000/90000 [00:00<00:00, 107009.77it/s]


In [8]:
df['tags'] = df['tags'].apply(lambda x: ' '.join(x))

In [9]:
all_tag

['PER_B',
 'DAT_B',
 '-',
 'ORG_B',
 'CVL_B',
 'NUM_B',
 'LOC_B',
 'EVT_B',
 'TRM_B',
 'TRM_I',
 'EVT_I',
 'PER_I',
 'CVL_I',
 'NUM_I',
 'TIM_B',
 'TIM_I',
 'ANM_B',
 'DAT_I',
 'FLD_B',
 'ORG_I',
 'MAT_B',
 'MAT_I',
 'AFW_B',
 'LOC_I',
 'AFW_I',
 'PLT_B',
 'FLD_I',
 'ANM_I',
 'PLT_I']

In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 학습 데이터 개수와 테스트 데이터 개수 출력하기
print(f'train data size: {len(train_df)}, test data size: {len(test_df)}')

train data size: 72000, test data size: 18000


In [11]:
train_data_sentence = [sent.split() for sent in train_df['text'].values ]
test_data_sentence = [sent.split() for sent in test_df['text'].values]
train_data_label = [tag.split() for tag in train_df['tags'].values]
test_data_label = [tag.split() for tag in test_df['tags'].values]

In [12]:
print(train_data_sentence[2])
print(train_data_label[2])

['타석에', '서서', '힘', '일체', '안들이고', '좌', '·', '우', '외야로', '타호를', '가볍게', '보내', '율동을', '확인하는데', '그치더라', '．']
['-', '-', '-', '-', '-', 'TRM_B', '-', 'TRM_B', '-', '-', '-', '-', '-', '-', '-', '-']


In [13]:
tag_to_index = {tag: index for index, tag in enumerate(all_tag)}
index_to_tag = {index: tag for index, tag in enumerate(all_tag)}

In [14]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 29


In [15]:
tokenizer =BertTokenizer.from_pretrained('klue/bert-base')
sent = train_data_sentence[1]
label = train_data_label[1]

print(' 문장 :', sent)
print(' 레이블 :',label)
print(' 레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

 문장 : ['김준호', '트레이닝복은', '“소비자들이', '새로운', 'HD방식', '서드샷용', '화면', '사진기를', '활용하기', '시작하면', '큰', '광대역', '서브를', '원하게', '될', '것”이라고', '인터넷', '업체', '창설', '검토', '후경을', '설명했다', '.']
 레이블 : ['PER_B', 'CVL_B', 'CVL_B', '-', 'TRM_B', 'TRM_B', 'TRM_I', 'TRM_I', '-', '-', '-', '-', '-', '-', '-', '-', 'TRM_B', '-', '-', '-', '-', '-', '-']
 레이블의 정수 인코딩 : [0, 4, 4, 2, 8, 8, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2]
문장의 길이 : 23
레이블의 길이 : 23


In [16]:
tokens = []
for one_word in sent:
#각 단어에 대해서 서브워드로 분리.
# ex) one_word = '쿠 마 리' ===> subword_tokens = ['쿠', '##마 리']
# ex) one_word = '한 동 수 가' ===> subword_tokens = ['한 동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('BERT 토크나이저 적용 후 문장 :',tokens)
print(' 레이블 :', label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

BERT 토크나이저 적용 후 문장 : ['김준', '##호', '트레이닝', '##복', '##은', '“', '소비자', '##들이', '새로운', 'HD', '##방식', '서', '##드', '##샷', '##용', '화면', '사진', '##기', '##를', '활용', '##하기', '시작', '##하면', '큰', '광대', '##역', '서브', '##를', '원하', '##게', '될', '것', '”', '이라', '##고', '인터넷', '업체', '창설', '검토', '후', '##경', '##을', '설명', '##했', '##다', '.']
 레이블 : ['PER_B', 'CVL_B', 'CVL_B', '-', 'TRM_B', 'TRM_B', 'TRM_I', 'TRM_I', '-', '-', '-', '-', '-', '-', '-', '-', 'TRM_B', '-', '-', '-', '-', '-', '-']
레이블의 정수 인코딩 : [0, 4, 4, 2, 8, 8, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2]
문장의 길이 : 46
레이블의 길이 : 23


In [17]:
tokens = []
labels_ids = []
for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens)- 1))

print('토 큰 화 후 문 장 :',tokens)
print('레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩 :',labels_ids)
print('문 장 의 길 이 :', len(tokens))
print('레 이 블 의 길 이 :', len(labels_ids))

토 큰 화 후 문 장 : ['김준', '##호', '트레이닝', '##복', '##은', '“', '소비자', '##들이', '새로운', 'HD', '##방식', '서', '##드', '##샷', '##용', '화면', '사진', '##기', '##를', '활용', '##하기', '시작', '##하면', '큰', '광대', '##역', '서브', '##를', '원하', '##게', '될', '것', '”', '이라', '##고', '인터넷', '업체', '창설', '검토', '후', '##경', '##을', '설명', '##했', '##다', '.']
레 이 블 : ['PER_B', '[PAD]', 'CVL_B', '[PAD]', '[PAD]', 'CVL_B', '[PAD]', '[PAD]', '-', 'TRM_B', '[PAD]', 'TRM_B', '[PAD]', '[PAD]', '[PAD]', 'TRM_I', 'TRM_I', '[PAD]', '[PAD]', '-', '[PAD]', '-', '[PAD]', '-', '-', '[PAD]', '-', '[PAD]', '-', '[PAD]', '-', '-', '[PAD]', '[PAD]', '[PAD]', 'TRM_B', '-', '-', '-', '-', '[PAD]', '[PAD]', '-', '[PAD]', '[PAD]', '-']
레이블의 정수 인코딩 : [0, -100, 4, -100, -100, 4, -100, -100, 2, 8, -100, 8, -100, -100, -100, 9, 9, -100, -100, 2, -100, 2, -100, 2, 2, -100, 2, -100, 2, -100, 2, 2, -100, -100, -100, 8, 2, 2, 2, 2, -100, -100, 2, -100, -100, 2]
문 장 의 길 이 : 46
레 이 블 의 길 이 : 46


In [18]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0,pad_token_id_for_label=-100):
  pad_token_id = tokenizer.pad_token_id
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):
      #하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      #서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
      labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))
      #[CLS]와[SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len-2의 길이로 변환.
      #ex)max_seq_len=64라면 길이가62보다 긴 샘플은 뒷 부분을 자르고 길 이62로 변환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
        tokens = tokens[:(max_seq_len - special_tokens_count)]
        labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
    #[SEP]를 추가하는 코드
    #1.토큰화 결과의 맨 뒷 부분에[SEP]토큰 추가 #2. 레이블에도 맨 뒷 부분에 -100 추가.
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]
    #[CLS]를 추가하는 코드
    #1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
    #2. 레이블의 맨 앞 부분에도 -100 추가.
    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids
    #정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    #어텐션 마스크 생성
    attention_mask = [1] * len(input_id)
    #정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)
    #정수 인코딩,어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    #세그먼트 인코딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩.(단, 이 경우는 패딩 토큰의 ID가 -100)
    label = labels_ids + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)
  
  return (input_ids, attention_masks, token_type_ids), data_labels

In [19]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 18000/18000 [00:26<00:00, 675.86it/s]


In [20]:
print('기 존 원 문 :', train_data_sentence[0])
print('기 존 레 이 블 :', train_data_label[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토 큰 화 후 레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('-' * 50)
print('정 수 인 코 딩 결 과 :', X_train[0][0])
print('정수 인코딩 레이블:',y_train[0])

기 존 원 문 : ['3일', '몽모렌시폭포', '소공동', '롯데호텔에서', '알바레스', '부총리', '겸', '한러정상회담', '별미국수', '체재로', '제11회', '과학기술관계장관회의가', '열렸다', '.']
기 존 레 이 블 : ['DAT_B', 'LOC_B', 'LOC_B', 'AFW_B', 'PER_B', 'CVL_B', '-', 'ORG_B', 'CVL_B', '-', 'NUM_B', 'EVT_B', '-', '-']
--------------------------------------------------
토 큰 화 후 원 문 : ['[CLS]', '3', '##일', '몽', '##모', '##렌', '##시', '##폭', '##포', '소공동', '롯데호텔', '##에서', '알바', '##레스', '부총리', '겸', '한', '##러', '##정', '##상', '##회', '##담', '별미', '##국수', '체', '##재', '##로', '제', '##11', '##회', '과학', '##기술', '##관계', '##장', '##관', '##회의', '##가', '열렸', '##다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [21]:
print('세그먼트 인코딩 :', X_train[2][0])
print('어텐션 마스크 :', X_train[1][0])

세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [32]:
class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                        name='classifier')
  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
    # 전체 시퀀스에 대해서 분류해야 하므로 outputs[0]임에 주의
    all_output = outputs[0]
    prediction = self.classifier(all_output)
    return prediction

In [33]:
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15],[0.4, 0.5, 0.1]]])

In [34]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [35]:
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2]) ), active_loss)
print(reduced_logits)

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [36]:
labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


In [37]:
def compute_loss(labels, logits):
  # 다중 클래스 분류 문제에서 소프트맥스 함수 미사용 시 from_logits=True로 설정

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  #-100의 값을 가진 정수에 대해서는 오차를 반영하지 않도록labels를 수정.
  active_loss = tf.reshape(labels, (-1,)) != -100

  # activa_loss 로 부 터 reduced_logits 과 labels 를 각 각 얻 는 다 .
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

In [38]:
#TPU작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [39]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [40]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
  def sequences_to_tags(self, label_ids, pred_ids):
    label_list = []
    pred_list = []
    for i in range(0, len(label_ids)):
      label_tag = []
      pred_tag = []
      for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])
      label_list.append(label_tag)
      pred_list.append(pred_tag)
    return label_list, pred_list 
    
  #에포크가 끝날 때마다 실행되는 함수}
  def on_epoch_end(self, epoch, logs={}):
    y_predicted = self.model.predict(self.X_test)
    y_predicted = np.argmax(y_predicted, axis = 2)
    label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
    score = f1_score(label_list, pred_list, suffix=True)
    print(' - f1: {:04.2f}'.format(score * 100))
    print(classification_report(label_list, pred_list, suffix=True))

In [45]:
f1_score_report = F1score(X_test, y_test)

model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3
   6/2250 [..............................] - ETA: 3:03 - loss: 0.1433

563/563 [==============================] - 13s 21ms/step


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: ORG_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: - seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: PER_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: CVL_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: LOC_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:167: Use

 - f1: 82.69
              precision    recall  f1-score   support

        AFW_       0.60      0.55      0.57       893
        ANM_       0.70      0.78      0.74      1277
        CVL_       0.83      0.82      0.83     11515
        DAT_       0.90      0.92      0.91      5097
        EVT_       0.75      0.77      0.76      2319
        FLD_       0.65      0.66      0.65       487
        LOC_       0.82      0.88      0.85      4223
        MAT_       0.77      0.20      0.32        49
        NUM_       0.91      0.92      0.92     11135
        ORG_       0.89      0.85      0.87      8224
        PER_       0.86      0.91      0.88      8562
        PLT_       0.58      0.25      0.35        57
        TIM_       0.82      0.92      0.87       639
        TRM_       0.75      0.73      0.74      3750
           _       0.77      0.77      0.77     25426

   micro avg       0.82      0.83      0.83     83653
   macro avg       0.77      0.73      0.74     83653
weighted avg 

In [53]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                                pad_token_id_for_segment=0,pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id
  input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []
  for example in tqdm(examples):
    tokens = []
    label_mask = []
    for one_word in example:
      #하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드를 제외하고 그 뒤의 서브워드들은 -100으로 채운다.
      label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))
      #[CLS]와[SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len-2의 길이로 변환.
      #ex)max_seq_len=64라면 길이가62보다 긴 샘플은 뒷 부분을 자르고 길 이62로 변환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      label_mask = label_mask[:(max_seq_len - special_tokens_count)]

    #[SEP]를 추가하는 코드
    #1.토큰화 결과의 맨 뒷 부분에[SEP]토큰 추가
    #2. 레이블에도 맨 뒷 부분에 -100 추가.
    tokens += [sep_token]
    label_mask += [pad_token_id_for_label]

    #[CLS]를 추가하는 코드
    #1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
    #2. 레이블의 맨 앞 부분에도 -100 추가.
    tokens = [cls_token] + tokens
    label_mask = [pad_token_id_for_label] + label_mask

    #정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    #어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

    #정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

    #정수 인코딩,어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    #세그먼트인코딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩.(단, 이 경우는 패딩 토큰의 ID가 -100)
    label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    label_masks.append(label_mask)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  label_masks = np.asarray(label_masks, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), label_masks

In [54]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 312.40it/s]


In [55]:
print('기 존 원 문 :', test_data_sentence[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레 이 블 마 스 크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기 존 원 문 : ['[마이데일리', '=', '마일영', '매사추세츠주지사]', '좌해', '인간문화재가', '백고천난', '끝에', '강호', '스트라스부르를', '꺾고', '오륜대회', '서울대회', '진출의', '실낱같은', '체념을', '되살렸다', '.']
--------------------------------------------------
토 큰 화 후 원 문 : ['[CLS]', '[', '마이', '##데', '##일리', '=', '마일', '##영', '매사추세츠', '##주', '##지사', ']', '좌', '##해', '인간', '##문화재', '##가', '백', '##고', '##천', '##난', '끝', '##에', '강호', '스트라', '##스', '##부', '##르', '##를', '꺾', '##고', '오', '##륜', '##대', '##회', '서울대', '##회', '진출', '##의', '실', '##낱', '##같', '##은', '체념', '##을', '되살', '##렸', '##다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [56]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples,max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)
  pred_list = []
  result_list = []
  for i in range(0, len(label_masks)):
    pred_tag = []
    #ex) 모델의 예측값 디코딩 과정
    # 예측값(y_predicted)에서 레이블 마스크(label_masks)의 값이 -100인 동일 위치의값을삭제
    # label_masks : [-100 0 -100 0 -100]
    #y_predicted:[ 0 1 0 2 0]==>[12]==> 최종 예측(pred_tag):[ PER-B PER-I]
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)
  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)
  
  return result_list

In [57]:
sent1='오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2='하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

test_samples = [sent1, sent2]
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer )
result_list

100%|██████████| 2/2 [00:00<00:00, 339.17it/s]


1/1 [==============================] - 1s 1s/step


[[('오리온스는', 'ORG_B'),
  ('리그', '-'),
  ('최정상급', '-'),
  ('포인트가드', 'CVL_B'),
  ('김동훈을', 'PER_B'),
  ('앞세우는', '-'),
  ('빠른', '-'),
  ('공수전환이', '-'),
  ('돋보이는', '-'),
  ('팀이다', '-')],
 [('하이신사에', 'LOC_B'),
  ('속한', '-'),
  ('섬들도', '-'),
  ('위로', '-'),
  ('솟아', '-'),
  ('있는데', '-'),
  ('타인은', '-'),
  ('살고', '-'),
  ('있어요', '-')]]